In [1]:
import math
import numpy as np
import scipy.sparse as sp
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

# 1. 학습 설정

In [2]:
config = {
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    'model_path' : "../model",


    'submission_path' : "../submission",
    'submission_name' : 'Ensembel_v3_submission.csv',

    'candidate_item_num' : 50,
    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

# 2. 데이터 전처리

In [3]:
class MakeMatrixDataSet():
    """
    MatrixDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'train_ratings.csv'))
        
        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('item')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('user')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['item'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['user'].apply(lambda x : self.user_encoder[x])

        self.user_train, self.user_valid = self.generate_sequence_data()

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['time']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def make_matrix(self, user_list, train = True):
        """
        user_item_dict를 바탕으로 행렬 생성
        """
        mat = torch.zeros(size = (user_list.size(0), self.num_item))
        for idx, user in enumerate(user_list):
            if train:
                mat[idx, self.user_train[user.item()]] = 1
            else:
                mat[idx, self.user_train[user.item()] + self.user_valid[user.item()]] = 1
        return mat

    def make_sparse_matrix(self, test = False):
        X = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        
        for user in self.user_train.keys():
            item_list = self.user_train[user]
            X[user, item_list] = 1.0
        
        if test:
            for user in self.user_valid.keys():
                item_list = self.user_valid[user]
                X[user, item_list] = 1.0

        return X.tocsr()

In [4]:
class AEDataSet(Dataset):
    def __init__(self, num_user):
        self.num_user = num_user
        self.users = [i for i in range(num_user)]

    def __len__(self):
        return self.num_user

    def __getitem__(self, idx): 
        user = self.users[idx]
        return torch.LongTensor([user])

# 3. 모델

In [5]:
class EASE():
    def __init__(self, X, reg):
        self.X = self._convert_sp_mat_to_sp_tensor(X)
        self.reg = reg
    
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res
    
    def fit(self):
        '''

        진짜 정말 간단한 식으로 모델을 만듬

        '''
        G = self.X.to_dense().t() @ self.X.to_dense()
        diagIndices = torch.eye(G.shape[0]) == 1
        G[diagIndices] += self.reg

        P = G.inverse()
        B = P / (-1 * P.diag())
        B[diagIndices] = 0

        self.pred = self.X.to_dense() @ B

In [6]:
def swish(x):
    return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
    return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())

class CompositePrior(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
        super(CompositePrior, self).__init__()
        
        self.mixture_weights = mixture_weights
        
        self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.mu_prior.data.fill_(0)
        
        self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_prior.data.fill_(0)
        
        self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_uniform_prior.data.fill_(10)
        
        self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
        self.encoder_old.requires_grad_(False)
        
    def forward(self, x, z):

        post_mu, post_logvar = self.encoder_old(x, dropout_rate = 0)

        stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
        post_prior = log_norm_pdf(z, post_mu, post_logvar)
        unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)
        
        gaussians = [stnd_prior, post_prior, unif_prior]
        gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]

        density_per_gaussian = torch.stack(gaussians, dim=-1)

        return torch.logsumexp(density_per_gaussian, dim=-1)

    
class Encoder(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
        super(Encoder, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, dropout_rate):
        norm = x.pow(2).sum(dim=-1).sqrt()
        x = x / norm[:, None]
    
        x = F.dropout(x, p=dropout_rate, training=self.training)
        
        h1 = self.ln1(swish(self.fc1(x)))
        h2 = self.ln2(swish(self.fc2(h1) + h1))
        h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
        h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
        h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
        return self.fc_mu(h5), self.fc_logvar(h5)


class RecVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim = 600, latent_dim = 200):
        super(RecVAE, self).__init__()

        self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
        self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, user_ratings, beta=None, gamma=0.0005, dropout_rate=0.7, calculate_loss=True):
        mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)    
        z = self.reparameterize(mu, logvar)
        x_pred = self.decoder(z)

        if calculate_loss:
            if gamma:
                norm = user_ratings.sum(dim=-1)
                kl_weight = gamma * norm
            elif beta:
                kl_weight = beta

            mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
            kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
            negative_elbo = -(mll - kld)
            
            return (mll, kld), negative_elbo
            
        else:
            return x_pred

    def update_prior(self):
        self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))

# 4. 학습 함수

In [7]:
def get_ndcg(pred_list, true_list):
    idcg = sum((1 / np.log2(rank + 2) for rank in range(1, len(pred_list))))
    dcg = 0
    for rank, pred in enumerate(pred_list):
        if pred in true_list:
            dcg += 1 / np.log2(rank + 2)
    ndcg = dcg / idcg
    return ndcg

# hit == recall == precision
def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit


def evaluate(model1, model2, RecVAE, X, user_train, user_valid, candidate_cnt):
    RecVAE.eval()

    mat = torch.from_numpy(X)

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    recon_mat1 = model1.pred.cpu()
    recon_mat1[mat == 1] = -np.inf
    rec_list1 = recon_mat1.argsort(dim = 1)

    recon_mat2 = model2.pred.T.cpu()
    recon_mat2[mat == 1] = -np.inf
    rec_list2 = recon_mat2.argsort(dim = 1)

    recon_mat3 = RecVAE(mat.to(device), calculate_loss = False).cpu().detach()
    recon_mat3[mat == 1] = -np.inf
    rec_list3 = recon_mat3.argsort(dim = 1)

    score_li = np.array([1 / np.log2(rank + 2) for rank in range(0, candidate_cnt)])

    for user, (rec1, rec2, rec3) in tqdm(enumerate(zip(rec_list1, rec_list2, rec_list3))):
        uv = user_valid[user]

        # ranking
        rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

        items = list(set(rec1 + rec2 + rec3))

        movie_df = pd.DataFrame(index = items)
        movie_df.loc[rec1, 'rec1_score'] = score_li
        movie_df.loc[rec2, 'rec2_score'] = score_li
        movie_df.loc[rec3, 'rec3_score'] = score_li
        movie_df = movie_df.fillna(min(score_li))
        movie_df['total_score'] = movie_df['rec1_score'] + movie_df['rec2_score'] + movie_df['rec3_score']
        movie_df = movie_df.sort_values('total_score', ascending = False)
        up = movie_df.index.tolist()[:10]

        NDCG += get_ndcg(pred_list = up, true_list = uv)
        HIT += get_hit(pred_list = up, true_list = uv)

    NDCG /= len(user_train)
    HIT /= len(user_train)

    return NDCG, HIT

def predict(model1, model2, RecVAE, X, candidate_cnt):
    RecVAE.eval()

    user2rec = {}

    mat = torch.from_numpy(X)

    recon_mat1 = model1.pred.cpu()
    recon_mat1[mat == 1] = -np.inf
    rec_list1 = recon_mat1.argsort(dim = 1)

    recon_mat2 = model2.pred.T.cpu()
    recon_mat2[mat == 1] = -np.inf
    rec_list2 = recon_mat2.argsort(dim = 1)

    recon_mat3 = RecVAE(mat.to(device), calculate_loss = False).cpu().detach()
    recon_mat3[mat == 1] = -np.inf
    rec_list3 = recon_mat3.argsort(dim = 1)

    score_li = np.array([1 / np.log2(rank + 2) for rank in range(0, candidate_cnt)])

    for user, (rec1, rec2, rec3) in tqdm(enumerate(zip(rec_list1, rec_list2, rec_list3))):
        # ranking
        rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

        items = list(set(rec1 + rec2 + rec3))

        movie_df = pd.DataFrame(index = items)
        movie_df.loc[rec1, 'rec1_score'] = score_li
        movie_df.loc[rec2, 'rec2_score'] = score_li
        movie_df.loc[rec3, 'rec3_score'] = score_li
        movie_df = movie_df.fillna(min(score_li))
        movie_df['total_score'] = movie_df['rec1_score'] + movie_df['rec2_score'] + movie_df['rec3_score']
        movie_df = movie_df.sort_values('total_score', ascending = False)
        up = movie_df.index.tolist()[:10]

        user2rec[user] = up

    return user2rec


from sklearn.linear_model import LogisticRegression


# def evaluate(model1, model2, RecVAE, X_df, y_df, X, user_train, user_valid, candidate_cnt):
#     RecVAE.eval()

#     mat = torch.from_numpy(X)

#     NDCG = 0.0 # NDCG@10
#     HIT = 0.0 # HIT@10

#     recon_mat1 = model1.pred.cpu()
#     copy_recon_mat1 = deepcopy(recon_mat1)
#     recon_mat1[mat == 1] = -np.inf
#     rec_list1 = recon_mat1.argsort(dim = 1)

#     recon_mat2 = model2.pred.T.cpu()
#     copy_recon_mat2 = deepcopy(recon_mat2)
#     recon_mat2[mat == 1] = -np.inf
#     rec_list2 = recon_mat2.argsort(dim = 1)

#     recon_mat3 = RecVAE(mat.to(device), calculate_loss = False).cpu().detach()
#     copy_recon_mat3 = deepcopy(recon_mat3.tanh())
#     recon_mat3[mat == 1] = -np.inf
#     rec_list3 = recon_mat3.argsort(dim = 1)

#     for user, (rec1, rec2, rec3) in tqdm(enumerate(zip(rec_list1, rec_list2, rec_list3))):

#         cif = LogisticRegression(random_state = config.seed).fit(X_df[6807 * user : 6807 * (user + 1), 1:], y_df[6807 * user : 6807 * (user + 1)])

#         uv = user_valid[user]

#         rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
#         rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
#         rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

#         items = list(set(rec1 + rec2 + rec3))
#         score_li = np.array([[copy_recon_mat1.numpy()[user][item], copy_recon_mat2.numpy()[user][item], copy_recon_mat3.numpy()[user][item]] for item in items])
#         score_li = cif.predict_proba(score_li)[:, 1]
        
#         movie_df = pd.DataFrame(index = items)
#         movie_df.loc[items, 'total_score'] = score_li
#         movie_df = movie_df.sort_values('total_score', ascending = False)
#         up = movie_df.index.tolist()[:10]

#         NDCG += get_ndcg(pred_list = up, true_list = uv)
#         HIT += get_hit(pred_list = up, true_list = uv)

#     NDCG /= len(user_train)
#     HIT /= len(user_train)

#     return NDCG, HIT


# def predict(model1, model2, RecVAE, X_df, y_df, X, candidate_cnt):
#     RecVAE.eval()

#     user2rec = {}

#     neg = torch.from_numpy(1 - X)
#     pos = torch.from_numpy(X)

#     recon_mat1 = model1.pred.cpu()
#     score1 = recon_mat1 * neg
#     rec_list1 = score1.argsort(dim = 1)

#     recon_mat2 = model2.pred.T.cpu()
#     score2 = recon_mat2 * neg
#     rec_list2 = score2.argsort(dim = 1)

#     recon_mat3 = RecVAE(pos.to(device), calculate_loss = False).cpu().detach()
#     score3 = recon_mat3 * neg
#     rec_list3 = score3.argsort(dim = 1)

#     for user, (rec1, rec2, rec3) in tqdm(enumerate(zip(rec_list1, rec_list2, rec_list3))):

#         cif = LogisticRegression(random_state = config.seed).fit(X_df[6807 * user : 6807 * (user + 1), 1:], y_df[6807 * user : 6807 * (user + 1)])

#         rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
#         rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
#         rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

#         items = list(set(rec1 + rec2 + rec3))
#         score_li = np.array([[recon_mat1.numpy()[user][item], recon_mat2.numpy()[user][item], recon_mat3.numpy()[user][item]] for item in items])
#         score_li = cif.predict_proba(score_li)[:, 1]
        
#         movie_df = pd.DataFrame(index = items)
#         movie_df.loc[items, 'total_score'] = score_li
#         movie_df = movie_df.sort_values('total_score', ascending = False)
#         up = movie_df.index.tolist()[:10]
        
#         user2rec[user] = up

#     return user2rec


def total_evaluate(model1, model2, RecVAE, X, user_train, user_valid, candidate_cnt):
    RecVAE.eval()
    
    rec1_NDCG = 0.0 # NDCG@10
    rec1_HIT = 0.0 # HIT@10

    rec2_NDCG = 0.0 # NDCG@10
    rec2_HIT = 0.0 # HIT@10

    rec3_NDCG = 0.0 # NDCG@10
    rec3_HIT = 0.0 # HIT@10

    rec12_NDCG = 0.0 # NDCG@10
    rec12_HIT = 0.0 # HIT@10

    rec13_NDCG = 0.0 # NDCG@10
    rec13_HIT = 0.0 # HIT@10

    rec23_NDCG = 0.0 # NDCG@10
    rec23_HIT = 0.0 # HIT@10

    rec123_NDCG = 0.0 # NDCG@10
    rec123_HIT = 0.0 # HIT@10

    mat = torch.from_numpy(X)

    recon_mat1 = model1.pred.cpu()
    recon_mat1[mat == 1] = -np.inf
    rec_list1 = recon_mat1.argsort(dim = 1)

    recon_mat2 = model2.pred.T.cpu()
    recon_mat2[mat == 1] = -np.inf
    rec_list2 = recon_mat2.argsort(dim = 1)

    recon_mat3 = RecVAE(mat.to(device), calculate_loss = False).cpu().detach()
    recon_mat3[mat == 1] = -np.inf
    rec_list3 = recon_mat3.argsort(dim = 1)
    
    for user, (rec1, rec2, rec3) in enumerate(zip(rec_list1, rec_list2, rec_list3)):
        uv = user_valid[user]

        # ranking
        rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        
        rec12 = list(set(rec1 + rec2))
        rec13 = list(set(rec1 + rec3))
        rec23 = list(set(rec2 + rec3))

        rec123 = list(set(rec1 + rec2 + rec3))

        rec1_NDCG += get_ndcg(pred_list = rec1, true_list = uv)
        rec1_HIT += get_hit(pred_list = rec1, true_list = uv)

        rec2_NDCG += get_ndcg(pred_list = rec2, true_list = uv)
        rec2_HIT += get_hit(pred_list = rec2, true_list = uv)

        rec3_NDCG += get_ndcg(pred_list = rec3, true_list = uv)
        rec3_HIT += get_hit(pred_list = rec3, true_list = uv)


        rec12_NDCG += get_ndcg(pred_list = rec12, true_list = uv)
        rec12_HIT += get_hit(pred_list = rec12, true_list = uv)

        rec13_NDCG += get_ndcg(pred_list = rec13, true_list = uv)
        rec13_HIT += get_hit(pred_list = rec13, true_list = uv)

        rec23_NDCG += get_ndcg(pred_list = rec23, true_list = uv)
        rec23_HIT += get_hit(pred_list = rec23, true_list = uv)


        rec123_NDCG += get_ndcg(pred_list = rec123, true_list = uv)
        rec123_HIT += get_hit(pred_list = rec123, true_list = uv)

    print(f'rec1 | NDCG@10: {rec1_NDCG / len(user_train):.5f}| HIT@10: {rec1_HIT / len(user_train):.5f}')
    print(f'rec2 | NDCG@10: {rec2_NDCG / len(user_train):.5f}| HIT@10: {rec2_HIT / len(user_train):.5f}')
    print(f'rec3 | NDCG@10: {rec3_NDCG / len(user_train):.5f}| HIT@10: {rec3_HIT / len(user_train):.5f}')

    print(f'rec12 | NDCG@10: {rec12_NDCG / len(user_train):.5f}| HIT@10: {rec12_HIT / len(user_train):.5f}')
    print(f'rec13 | NDCG@10: {rec13_NDCG / len(user_train):.5f}| HIT@10: {rec13_HIT / len(user_train):.5f}')
    print(f'rec23 | NDCG@10: {rec23_NDCG / len(user_train):.5f}| HIT@10: {rec23_HIT / len(user_train):.5f}')

    print(f'rec123 | NDCG@10: {rec123_NDCG / len(user_train):.5f}| HIT@10: {rec123_HIT / len(user_train):.5f}')

# 5. 학습

In [8]:
make_matrix_data_set = MakeMatrixDataSet(config = config)
user_train, user_valid = make_matrix_data_set.get_train_valid_data()
X = make_matrix_data_set.make_sparse_matrix()

In [9]:
model1 = EASE(X = X, reg = 750)
model1.fit()

In [10]:
model2 = EASE(X = X.T, reg = 4400)
model2.fit()

In [11]:
model3 = RecVAE(
    input_dim = make_matrix_data_set.num_item,).to(device)

model3.load_state_dict(torch.load(os.path.join(config.model_path, 'RecVAE_v3.pt')))

<All keys matched successfully>

In [13]:
total_evaluate(
    model1 = model1, 
    model2 = model2, 
    RecVAE = model3,
    X = X.todense(),
    user_train = user_train, 
    user_valid = user_valid, 
    candidate_cnt = 10)

rec1 | NDCG@10: 0.31055| HIT@10: 0.20384
rec2 | NDCG@10: 0.30428| HIT@10: 0.20021
rec3 | NDCG@10: 0.28574| HIT@10: 0.19214
rec12 | NDCG@10: 0.23462| HIT@10: 0.22666
rec13 | NDCG@10: 0.21648| HIT@10: 0.24872
rec23 | NDCG@10: 0.21378| HIT@10: 0.24704
rec123 | NDCG@10: 0.20457| HIT@10: 0.26235


In [16]:
users = np.concatenate([np.repeat(i, 6807) for i in range(31360)])

model1_score = model1.pred.sigmoid().cpu().numpy().reshape(-1)
model2_score = model2.pred.T.sigmoid().cpu().numpy().reshape(-1)
model3_score = model3(torch.from_numpy(X.todense()).to(device), calculate_loss = False).tanh().cpu().detach().numpy().reshape(-1)

X_df = np.concatenate([users.reshape(-1, 1), model1_score.reshape(-1, 1), model2_score.reshape(-1, 1), model3_score.reshape(-1, 1)], axis = 1)
y_df = X.toarray().reshape(-1)

In [13]:
# genre = pd.read_csv(os.path.join(config.data_path, 'genres.tsv'), sep='\t')
# genre['genres'] = 1
# genre['item_idx'] = genre['item'].apply(lambda x : make_matrix_data_set.item_encoder[x])
# genre = pd.pivot_table(genre, values='genres', index=['item_idx'], columns=['genre'], aggfunc=np.sum, fill_value=0).values

In [17]:
candidate_cnt = 10

ndcg, hit = evaluate(
                model1 = model1, 
                model2 = model2, 
                RecVAE = model3,
                X_df = X_df, 
                y_df = y_df,
                X = X.todense(),
                user_train = user_train, 
                user_valid = user_valid, 
                candidate_cnt = candidate_cnt)

print(f'candidate_cnt: {candidate_cnt}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

31360it [12:50, 40.70it/s]

candidate_cnt: 10| NDCG@10: 0.31253| HIT@10: 0.20492


In [ ]:
for candidate_cnt in [5 * i for i in range(2, 21)]:
    
    ndcg, hit = evaluate(
                model1 = model1, 
                model2 = model2, 
                RecVAE = model3,
                X = X.todense(),
                user_train = user_train, 
                user_valid = user_valid, 
                candidate_cnt = candidate_cnt)

    print(f'candidate_cnt: {candidate_cnt}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

```
RecVAE-v3
candidate_cnt: 10| NDCG@10: 0.31412| HIT@10: 0.20595
candidate_cnt: 15| NDCG@10: 0.31573| HIT@10: 0.20745
candidate_cnt: 20| NDCG@10: 0.31644| HIT@10: 0.20807
candidate_cnt: 25| NDCG@10: 0.31687| HIT@10: 0.20851
candidate_cnt: 30| NDCG@10: 0.31696| HIT@10: 0.20863
candidate_cnt: 35| NDCG@10: 0.31708| HIT@10: 0.20872
candidate_cnt: 40| NDCG@10: 0.31712| HIT@10: 0.20878
candidate_cnt: 45| NDCG@10: 0.31717| HIT@10: 0.20882
candidate_cnt: 50| NDCG@10: 0.31723| HIT@10: 0.20886
candidate_cnt: 55| NDCG@10: 0.31723| HIT@10: 0.20884
candidate_cnt: 60| NDCG@10: 0.31726| HIT@10: 0.20887
candidate_cnt: 65| NDCG@10: 0.31729| HIT@10: 0.20888
candidate_cnt: 70| NDCG@10: 0.31727| HIT@10: 0.20887
candidate_cnt: 75| NDCG@10: 0.31730| HIT@10: 0.20889
candidate_cnt: 80| NDCG@10: 0.31730| HIT@10: 0.20890
candidate_cnt: 85| NDCG@10: 0.31733| HIT@10: 0.20888

```

```
RecVAE-v5
candidate_cnt: 10| NDCG@10: 0.31372| HIT@10: 0.20602
candidate_cnt: 15| NDCG@10: 0.31518| HIT@10: 0.20735
candidate_cnt: 20| NDCG@10: 0.31569| HIT@10: 0.20779
candidate_cnt: 25| NDCG@10: 0.31605| HIT@10: 0.20812
candidate_cnt: 30| NDCG@10: 0.31616| HIT@10: 0.20822
candidate_cnt: 35| NDCG@10: 0.31627| HIT@10: 0.20833
candidate_cnt: 40| NDCG@10: 0.31636| HIT@10: 0.20843
candidate_cnt: 45| NDCG@10: 0.31643| HIT@10: 0.20846
candidate_cnt: 50| NDCG@10: 0.31644| HIT@10: 0.20848
candidate_cnt: 55| NDCG@10: 0.31653| HIT@10: 0.20854
candidate_cnt: 60| NDCG@10: 0.31657| HIT@10: 0.20857
candidate_cnt: 65| NDCG@10: 0.31652| HIT@10: 0.20856
candidate_cnt: 70| NDCG@10: 0.31653| HIT@10: 0.20856
candidate_cnt: 75| NDCG@10: 0.31651| HIT@10: 0.20855
candidate_cnt: 80| NDCG@10: 0.31658| HIT@10: 0.20858
candidate_cnt: 85| NDCG@10: 0.31660| HIT@10: 0.20859
candidate_cnt: 90| NDCG@10: 0.31658| HIT@10: 0.20860
candidate_cnt: 95| NDCG@10: 0.31662| HIT@10: 0.20862
candidate_cnt: 100| NDCG@10: 0.31658| HIT@10: 0.20861

```

```
candidate_cnt: 10| NDCG@10: 0.31372| HIT@10: 0.20602
```

```
none + logistic

candidate_cnt: 10| NDCG@10: 0.31411| HIT@10: 0.20686
```

```
sigmoid + logistic

candidate_cnt: 10| NDCG@10: 0.31568| HIT@10: 0.20702
```

```
softmax + logistic

candidate_cnt: 10| NDCG@10: 0.28598| HIT@10: 0.19227
```

```
none + sum

candidate_cnt: 10| NDCG@10: 0.31381| HIT@10: 0.20633
```

```
sigmoid + sum

candidate_cnt: 10| NDCG@10: 0.31469| HIT@10: 0.20636
```

[147, 197, 188, 313, 228, 273, 189, 114, 42, 242]

```
non softmax

score  genre_score  total_score
197   0.902873     0.411256     1.314129
42    0.815624     0.411256     1.226880
933   0.981398     0.172427     1.153824
718   0.671753     0.402721     1.074474
484   0.923667     0.149443     1.073110
376   0.883676     0.173539     1.057215
667   0.919388     0.097644     1.017032
650   0.855481     0.151946     1.007427
2200  0.878635     0.125442     1.004077
714   0.719053     0.210817     0.929870
1844  0.698690     0.226953     0.925643
273   0.810516     0.097710     0.908226
760   0.763495     0.036601     0.800096
313   0.766302     0.027069     0.793371
777   0.641927     0.142652     0.784579
228   0.733693     0.044058     0.777750
734   0.738835     0.031143     0.769978
1354  0.438386     0.122450     0.560836

```


```
softmax


score  genre_score  total_score
197   0.053804     0.411256     0.465060
42    0.053799     0.411256     0.465055
718   0.053844     0.402721     0.456565
1844  0.053790     0.226953     0.280743
714   0.053817     0.210817     0.264634
376   0.053824     0.173539     0.227363
933   0.053912     0.172427     0.226339
650   0.053820     0.151946     0.205766
484   0.053827     0.149443     0.203270
777   0.053828     0.142652     0.196481
2200  0.053810     0.125442     0.179252
1354  0.053825     0.122450     0.176275
273   0.053800     0.097710     0.151510
667   0.053807     0.097644     0.151450
228   0.053814     0.044058     0.097871
760   0.053811     0.036601     0.090412
734   0.053804     0.031143     0.084947
313   0.053817     0.027069     0.080885

```

In [ ]:
for candidate_cnt in [5 * i for i in range(3, 21)]:
    ndcg, hit = evaluate(
            model1 = model1, 
            model2 = model2, 
            RecVAE = model3,
            X_df = X_df,
            y_df = y_df,
            X = X.todense(),
            user_train = user_train, 
            user_valid = user_valid, 
            candidate_cnt = candidate_cnt)

    print(f'candidate_cnt: {candidate_cnt}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

In [25]:
torch.FloatTensor([1, 2, 3, -np.inf]).argsort()

tensor([3, 0, 1, 2])

```
Logistic + 'l2'

candidate_cnt: 5| NDCG@10: 0.35450| HIT@10: 0.18291
candidate_cnt: 10| NDCG@10: 0.31411| HIT@10: 0.20686
candidate_cnt: 15| NDCG@10: 0.31367| HIT@10: 0.20634

```

```

log2(rank + 1)

candidate_cnt: 5| NDCG@10: 0.35405| HIT@10: 0.18237
candidate_cnt: 10| NDCG@10: 0.31403| HIT@10: 0.20607
candidate_cnt: 15| NDCG@10: 0.31528| HIT@10: 0.20717
candidate_cnt: 20| NDCG@10: 0.31590| HIT@10: 0.20775
candidate_cnt: 25| NDCG@10: 0.31634| HIT@10: 0.20822
candidate_cnt: 30| NDCG@10: 0.31650| HIT@10: 0.20833
candidate_cnt: 35| NDCG@10: 0.31657| HIT@10: 0.20840
candidate_cnt: 40| NDCG@10: 0.31667| HIT@10: 0.20850
candidate_cnt: 45| NDCG@10: 0.31669| HIT@10: 0.20852
candidate_cnt: 50| NDCG@10: 0.31672| HIT@10: 0.20852
candidate_cnt: 55| NDCG@10: 0.31676| HIT@10: 0.20859
candidate_cnt: 60| NDCG@10: 0.31678| HIT@10: 0.20856
candidate_cnt: 65| NDCG@10: 0.31674| HIT@10: 0.20855
candidate_cnt: 70| NDCG@10: 0.31675| HIT@10: 0.20855
candidate_cnt: 75| NDCG@10: 0.31673| HIT@10: 0.20856
candidate_cnt: 80| NDCG@10: 0.31677| HIT@10: 0.20855
candidate_cnt: 85| NDCG@10: 0.31676| HIT@10: 0.20856
candidate_cnt: 90| NDCG@10: 0.31679| HIT@10: 0.20856
candidate_cnt: 95| NDCG@10: 0.31678| HIT@10: 0.20857
candidate_cnt: 100| NDCG@10: 0.31677| HIT@10: 0.20856

```

# 6. 예측

In [8]:
make_matrix_data_set = MakeMatrixDataSet(config = config)
X_test = make_matrix_data_set.make_sparse_matrix(test = True)

In [9]:
model1 = EASE(X = X_test, reg = 750)
model1.fit()

In [10]:
model2 = EASE(X = X_test.T, reg = 4400)
model2.fit()

In [11]:
model3 = RecVAE(
    input_dim = make_matrix_data_set.num_item,).to(device)

model3.load_state_dict(torch.load(os.path.join(config.model_path, 'RecVAE_v3.pt')))

<All keys matched successfully>

In [12]:
# users = np.concatenate([np.repeat(i, 6807) for i in range(31360)])

# model1_score = model1.pred.cpu().numpy().reshape(-1)
# model2_score = model2.pred.T.cpu().numpy().reshape(-1)
# model3_score = model3(torch.from_numpy(X_test.todense()).to(device), calculate_loss = False).cpu().detach().numpy().reshape(-1)

# X_df = np.concatenate([users.reshape(-1, 1), model1_score.reshape(-1, 1), model2_score.reshape(-1, 1), model3_score.reshape(-1, 1)], axis = 1)
# y_df = X_test.toarray().reshape(-1)

In [12]:
user2rec_list = predict(
    model1 = model1, 
    model2 = model2, 
    RecVAE = model3,
    X = X_test.todense(),
    candidate_cnt = 50)

submision = []
users = [i for i in range(0, make_matrix_data_set.num_user)]
for user in users:
    rec_item_list = user2rec_list[user]
    for item in rec_item_list:
        submision.append(
            {   
                'user' : make_matrix_data_set.user_decoder[user],
                'item' : make_matrix_data_set.item_decoder[item],
            }
        )

submision = pd.DataFrame(submision)

31360it [01:46, 294.11it/s]


In [14]:
submision.to_csv(os.path.join(config.submission_path, config.submission_name), index=False)

In [15]:
submision

,user,item
0,11,4370
1,11,4886
2,11,40815
3,11,7373
4,11,8961
...,...,...
313595,138493,8961
313596,138493,110
313597,138493,5349
313598,138493,1022
